In [107]:
from bs4 import BeautifulSoup
import newspaper
from newspaper import Article
from textblob import TextBlob
from nltk.tokenize.punkt import PunktSentenceTokenizer
import networkx as nx
import numpy as np
from nltk.tokenize.punkt import PunktSentenceTokenizer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
import nltk.data
import urllib.request
from urllib.request import urlopen
import pymysql
import yaml

In [108]:
import tweepy
from tweepy import OAuthHandler
import re
from textblob import TextBlob
from newspaper import Article
from twython import Twython
##set up access to twitter website
consumer_key = 'BE7MnHVqk6bgXqdKMb0jGIbun'
consumer_secret = 'bWnvrCaAfrQlAgLM2gGgUwGvLefOsl41zMQf1kpUfvPTBXd41O'
access_token = '714916782142840834-I1zEtpMksHaqbqQYPynOPQxHbCNlyAJ'
access_secret = '8yqnE4DkVJ96Qm8OohyaszNcYj5JQEffB43ESIaeBcDj0'
 
#auth = OAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_token, access_secret)

twitter=Twython(consumer_key,consumer_secret)
auth=twitter.get_authentication_tokens()
OAUTH_TOKEN=auth['oauth_token']
OAUTH_TOKEN_SECRET=auth['oauth_token_secret']
auth['auth_url']

'https://api.twitter.com/oauth/authenticate?oauth_token=w3sNGQAAAAAAzGLWAAABW-toN34'

In [109]:

#api = tweepy.API(auth)

#function definitions
def clean_tweet(tweet):
        pattern = re.compile('[^A-Za-z0-9]+')
        tweet = pattern.sub(' ', tweet)
        #print encoded_tweet
       # et = pattern.sub(' ', tweet)
    #print encoded_tweet

        words = tweet.split()

    # Filter unnecessary words
        for w in words:
            if w.startswith("RT") or w.startswith("www") or w.startswith("http") or w.startswith("https"):
                words.remove(w)
        
    #code = str(words.encode('utf-8'))
   #     print(words)
        return str(words)
    
def get_tweet_sentiment(tweet):
        
        analysis = TextBlob(clean_tweet(tweet))
        if analysis.sentiment.polarity > 0.5:
            return 'excited'
        elif analysis.sentiment.polarity <= 0.5 and analysis.sentiment.polarity > 0 :
            return 'happy'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        
        elif analysis.sentiment.polarity > -0.5 and analysis.sentiment.polarity < 0:
            return 'sad'
        
        else:
            return 'morose'
tweets=[]
def get_tweets(query, count=50):
        
        
        
        try:
            fetched_tweets =api.search(q=query, count=count)
            for tweet in fetched_tweets:
                parsed_tweet={}
                parsed_tweet['text']=tweet.text
                parsed_tweet['sentiment']=get_tweet_sentiment(tweet.text)
            
                if tweet.retweet_count > 0:
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
            return tweets
        except tweepy.TweepError as e:
             print("Error : "+str(e))

def extract_entity_names(t):
    entity_names = []

    if hasattr(t, 'label') and t.label:
        if t.label() == 'NE':
            entity_names.append(' '.join([child[0] for child in t]))
        else:
            for child in t:
                entity_names.extend(extract_entity_names(child))

    return entity_names                

In [110]:
class Splitter(object):
    def __init__(self):
        self.nltk_splitter=nltk.data.load('tokenizers/punkt/english.pickle')
        self.nltk_tokenizer=nltk.tokenize.TreebankWordTokenizer()

    def split(self,text):
        sentences=self.nltk_splitter.tokenize(text)
        tokenized_sentences=[self.nltk_tokenizer.tokenize(sent) for sent in sentences]
        return tokenized_sentences

    
class POSTagger(object):
    def __init__(self):
        pass
    
    def pos_tag(self, sentences):
        pos=[nltk.pos_tag(sentence) for sentence in sentences]
        pos=[[(word,word,[postag]) for (word, postag) in sentence]for sentence in pos]
        return pos
    
class DictionaryTagger(object):
    def __init__(self, dictionary_paths):
        files = [open(path, 'r') for path in dictionary_paths]
        dictionaries = [yaml.load(dict_file) for dict_file in files]
        map(lambda x: x.close(), files)
        self.dictionary = {}
        self.max_key_size = 0
        for curr_dict in dictionaries:
            for key in curr_dict:
                if key in self.dictionary:
                    self.dictionary[key].extend(curr_dict[key])
                else:
                    self.dictionary[key] = curr_dict[key]
                    if isinstance(key,bool):
                        if key:
                            key = "True"
                        else:
                            key = "False"
                    self.max_key_size = max(self.max_key_size,len(key))

    def tag(self, postagged_sentences):
        return [self.tag_sentence(sentence) for sentence in postagged_sentences]

    def tag_sentence(self, sentence, tag_with_lemmas=False):
        """
        the result is only one tagging of all the possible ones.
        The resulting tagging is determined by these two priority rules:
            - longest matches have higher priority
            - search is made from left to right
        """
        tag_sentence = []
        N = len(sentence)
        if self.max_key_size == 0:
            self.max_key_size = N
        i = 0
        while (i < N):
            j = min(i + self.max_key_size, N) #avoid overflow
            tagged = False
            while (j > i):
                expression_form = ' '.join([word[0] for word in sentence[i:j]]).lower()
                expression_lemma = ' '.join([word[1] for word in sentence[i:j]]).lower()
                if tag_with_lemmas:
                    literal = expression_lemma
                else:
                    literal = expression_form
                if literal in self.dictionary:
                    #self.logger.debug("found: %s" % literal)
                    is_single_token = j - i == 1
                    original_position = i
                    i = j
                    taggings = [tag for tag in self.dictionary[literal]]
                    tagged_expression = (expression_form, expression_lemma, taggings)
                    if is_single_token: #if the tagged literal is a single token, conserve its previous taggings:
                        original_token_tagging = sentence[original_position][2]
                        tagged_expression[2].extend(original_token_tagging)
                    tag_sentence.append(tagged_expression)
                    tagged = True
                else:
                    j = j - 1
            if not tagged:
                tag_sentence.append(sentence[i])
                i += 1
        return tag_sentence

In [111]:
dicttagger = DictionaryTagger([ '/home/vaidehi/Documents/positive.yml', '/home/vaidehi/Documents/negative.yml', '/home/vaidehi/Documents/inc.yml', '/home/vaidehi/Documents/dec.yml'])

In [112]:


#dict_tagged_sentences = dicttagger.tag(pos_tagged_sentences)

def sentence_score(sentence_tokens, previous_token, acum_score):    
    if not sentence_tokens:
        return acum_score
    else:
        current_token = sentence_tokens[0]
        tags = current_token[2]
        token_score = sum([value_of(tag) for tag in tags])
        if previous_token is not None:
            previous_tags = previous_token[2]
            if 'inc' in previous_tags:
                token_score *= 2.0
            elif 'dec' in previous_tags:
                token_score /= 2.0
            elif 'inv' in previous_tags:
                token_score *= -1.0
        return sentence_score(sentence_tokens[1:], current_token, acum_score + token_score)
def value_of(sentiment):
    if sentiment=='positive':
        return 1.0
    if sentiment =='negative':
        return -1.0
    return 0.0
    
    
def sentiment_score(review):
    return sum([sentence_score(sentence, None, 0.0) for sentence in review])

def normalization(score):
    min_a=-20.0
    max_a=+20.0
    a=-1.0
    b=1.0
    x_norm= a + (score-min_a)*(b-a)/(max_a-min_a)
    return x_norm





In [124]:

world_news=[]

f = urlopen('http://timesofindia.indiatimes.com/world')
soup = BeautifulSoup(f.fp,"lxml")
for link in soup.select('.w_tle'):
    a="http://timesofindia.indiatimes.com/world"
    s=link.find('a').attrs['href']
    if 'http://timesofindia.indiatimes.com/world' in s:
        world_news.append(link.find('a').attrs['href'])
    elif 'photogallery' in s or 'video' in s or 'photostory' in s:
        continue
    else:
        world_news.append(a+s)
f = urlopen('http://indianexpress.com/section/world/')
soup = BeautifulSoup(f.fp,"lxml")
for link in soup.select('.short-info'):
      world_news.append(link.find('a').attrs['href'])
        
for link in soup.select('.story'):
     world_news.append(link.find('a').attrs['href'])
        
for link in soup.select('.title'):
     world_news.append(link.find('a').attrs['href'])
f = urlopen('http://www.hindustantimes.com/world-news/')
soup = BeautifulSoup(f.fp,"lxml")
for link in soup.select('.media-body'):
    world_news.append(link.find('a').attrs['href'])
for link in soup.select('.headingfive'):
    s=link.find('a').attrs['href']
    if 'photos' in s or 'video' in s:
        continue
    else:
        world_news.append(link.find('a').attrs['href'])


In [9]:
len(world_news)

145

In [115]:
national_news=[]

f = urlopen('http://indianexpress.com/section/india/')
soup = BeautifulSoup(f.fp,"lxml")
for link in soup.select('.short-info'):
    national_news.append(link.find('a').attrs['href'])
        
for link in soup.select('.story'):
    national_news.append(link.find('a').attrs['href'])
        
for link in soup.select('.title'):
    national_news.append(link.find('a').attrs['href'])

f = urlopen('http://www.hindustantimes.com/india-news/')
soup = BeautifulSoup(f.fp,"lxml")
for link in soup.select('.top-news-sec'):
        national_news.append(link.find('a').attrs['href'])
for link in soup.select('.headingfive'):
    s=link.find('a').attrs['href']
    if 'photos' in s or 'video' in s:
        continue
    else:
        national_news.append(link.find('a').attrs['href'])
for link in soup.select('.media-body'):
    national_news.append(link.find('a').attrs['href'])
f = urlopen('http://timesofindia.indiatimes.com/india')
soup = BeautifulSoup(f.fp,"lxml")
for link in soup.select('.clearFix'):
    a="http://timesofindia.indiatimes.com"
    s=link.find('a').attrs['href']
    if 'http://timesofindia.indiatimes.com' in s:
        national_news.append(link.find('a').attrs['href'])
    elif 'photogallery' in s or 'video' in s or 'photostory' in s:
        continue
    else:
        national_news.append(a+s)

In [79]:
entertainment_news=[]
f = urlopen('http://www.hindustantimes.com/entertainment/')
soup = BeautifulSoup(f.fp,"lxml")
for link in soup.select('.media-body'):
    s=link.find('a').attrs['href']
    if 'photos' in s or 'video' in s:
        continue
    else:
        entertainment_news.append(link.find('a').attrs['href'])
for link in soup.select('.random-heading'):
    s=link.find('a').attrs['href']
    if 'photos' in s or 'video' in s:
        continue
    else:
        entertainment_news.append(link.find('a').attrs['href'])
len(entertainment_news)

52

In [89]:
len(national_news)

116

In [70]:
lifestyle_news=[]
f = urlopen('http://www.hindustantimes.com/lifestyle/')
soup = BeautifulSoup(f.fp,"lxml")
for link in soup.select('.media-body'):
    s=link.find('a').attrs['href']
    if 'photos' in s or 'video' in s:
        continue
    else:
        lifestyle_news.append(link.find('a').attrs['href'])
for link in soup.select('.headingfive'):
    s=link.find('a').attrs['href']
    if 'photos' in s or 'video' in s:
        continue
    else:
        lifestyle_news.append(link.find('a').attrs['href'])
for link in soup.select('.random-heading'):
    s=link.find('a').attrs['href']
    if 'photos' in s or 'video' in s:
        continue
    else:
        lifestyle_news.append(link.find('a').attrs['href'])
len(lifestyle_news)

61

In [71]:
#Indian Express Technology
tech_news=[]
f = urlopen('http://indianexpress.com/section/technology/')
soup = BeautifulSoup(f.fp,"lxml")
for link in soup.select('.review-list'):
    s=link.find('a').attrs['href']
    if 'photos' in s or 'video' in s:
        continue
    else:
        tech_news.append(link.find('a').attrs['href'])
for link in soup.select('h3'):
    s=link.find('a').attrs['href']
    if 'photos' in s or 'video' in s:
        continue
    else:
        tech_news.append(link.find('a').attrs['href'])
for link in soup.select('.common-link'):
    s=link.find('a').attrs['href']
    if 'photos' in s or 'video' in s:
        continue
    else:
        tech_news.append(link.find('a').attrs['href'])
        
len(tech_news)

23

In [72]:
sports_news=[]
f = urlopen('http://timesofindia.indiatimes.com/sports')
soup = BeautifulSoup(f.fp,"lxml")
for link in soup.select('.list2'):
    a="http://timesofindia.indiatimes.com"
    s=link.find('a').attrs['href']
    if 'http://' in s:
        sports_news.append(link.find('a').attrs['href'])
    elif 'photogallery' in s or 'video' in s or 'photostory' in s:
        continue
    else:
        sports_news.append(a+s)
for link in soup.select('.main-story'):
    a="http://timesofindia.indiatimes.com"
    s=link.find('a').attrs['href']
    if 'http://' in s:
        sports_news.append(link.find('a').attrs['href'])
    elif 'photogallery' in s or 'video' in s or 'photostory' in s:
        continue
    else:
        sports_news.append(a+s)
f = urlopen('http://www.hindustantimes.com/sports-news/')
soup = BeautifulSoup(f.fp,"lxml")
for link in soup.select('.media-body'):
    s=link.find('a').attrs['href']
    if 'photos' in s or 'video' in s:
        continue
    else:
        sports_news.append(link.find('a').attrs['href'])
for link in soup.select('.headingfive'):
    s=link.find('a').attrs['href']
    if 'photos' in s or 'video' in s:
        continue
    else:
        sports_news.append(link.find('a').attrs['href'])
len(sports_news)

63

In [134]:
business_news=[]
f = urlopen('http://indianexpress.com/business/')
soup = BeautifulSoup(f.fp,"lxml")
for link in soup.select('.short-info'):
    business_news.append(link.find('a').attrs['href'])
        
for link in soup.select('.story'):
    business_news.append(link.find('a').attrs['href'])
        
for link in soup.select('.title'):
    business_news.append(link.find('a').attrs['href'])
f = urlopen('http://timesofindia.indiatimes.com/business')
soup = BeautifulSoup(f.fp,"lxml")
for link in soup.select('.w_tle'):
    a="http://timesofindia.indiatimes.com"
    s=link.find('a').attrs['href']
    if 'http://' in s:
        business_news.append(link.find('a').attrs['href'])
    elif 'photogallery' in s or 'video' in s or 'photostory' in s:
        continue
    else:
        business_news.append(a+s)

f = urlopen('http://www.hindustantimes.com/business-news/')
soup = BeautifulSoup(f.fp,"lxml")
for link in soup.select('.media-body'):
    s=link.find('a').attrs['href']
    if 'photos' in s or 'video' in s:
        continue
    else:
        business_news.append(link.find('a').attrs['href'])
for link in soup.select('.random-heading'):
    s=link.find('a').attrs['href']
    if 'photos' in s or 'video' in s:
        continue
    else:
        business_news.append(link.find('a').attrs['href'])
for link in soup.select('.headingfive'):
    s=link.find('a').attrs['href']
    if 'photos' in s or 'video' in s:
        continue
    else:
        business_news.append(link.find('a').attrs['href'])

In [60]:
len(business_news)

162

In [135]:
db=pymysql.connect("localhost","root","root","news_articles",charset="utf8")
cursor = db.cursor()
pic=cursor.execute('SELECT max(article_id) FROM articles ')
pic=cursor.fetchall()
lastnum=pic[0][0]
print(lastnum)

656


In [84]:
508

508

In [136]:

if lastnum == None:
    pic=1
else:
    pic=lastnum+1

for url in business_news:
      
    article=Article(url)
    article.download()
    try:
        article.parse()
    except newspaper.ArticleException:
        continue

    content = article.text
    splitter= Splitter()
    postagger = POSTagger()
    splitted_sentences=splitter.split(content)
    #print(splitted_sentences)
    pos_tagged_sentences = postagger.pos_tag(splitted_sentences)
    dict_tagged_sentences = dicttagger.tag(pos_tagged_sentences)
    #print(dict_tagged_sentences)
    norm_arti=normalization(sentiment_score(dict_tagged_sentences))

    print(norm_arti)
    
    #summary generation for each article
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    toz=sent_detector.tokenize(content.strip())
    bow_matrix = CountVectorizer(ngram_range=(1,3)).fit_transform(toz)
    normalized = TfidfTransformer().fit_transform(bow_matrix)
    bow_matrix_1 = CountVectorizer(stop_words={'english'},ngram_range=(2,3)).fit_transform(toz)
    normalized_1 = TfidfTransformer().fit_transform(bow_matrix_1)
    similarity_graph_1 = normalized_1 * normalized_1.T #sentence
    similarity_graph = normalized * normalized.T #sentence
    similarity_graph.toarray()
    nx_graph = nx.from_scipy_sparse_matrix(similarity_graph)
    scores = nx.pagerank(nx_graph)
    nx_graph_1 = nx.from_scipy_sparse_matrix(similarity_graph_1)
    scores_1 = nx.pagerank(nx_graph_1)
    sorted_list= sorted(([scores[i],s] for i,s in enumerate(toz[0:len(toz)])),
              reverse=True)
    summary=  [item[1] for item in sorted_list][0:3]
    s_x=''.join(summary)

    s_x
    
    #store the article headline
    y=article.title
    #download the main image of the article
    path="/home/vaidehi/FlaskApp/static/images/"
    
    
    w=article.top_image
    if w =='':
        storepath="p1.jpg"
    
    #try:
    else:
        
        urllib.request.urlretrieve(w, path+"i"+str(pic)+".jpg")
        pic=pic+1
        storepath="i"+str(pic)+".jpg" 

    #except urllib.URLError:
        #
    
    
    

    #print(y)
    #print(w)
    
    #determine the sentiments of the twitterverse on the news
    sentences = nltk.sent_tokenize(content)
    tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
    tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]
    chunked_sentences = nltk.ne_chunk_sents(tagged_sentences, binary=True)
    #tagged_sentences
    
    entity_names = []
    for tree in chunked_sentences:
    # Print results per sentence
    # print extract_entity_names(tree)
        entity_names.extend(extract_entity_names(tree))
    entity=set(entity_names)
    entities=[]
    for word in entity:
        entities.append(str(word))
    for entity in entities:
        results =twitter.search(q=entity, count=50)
        fetched_tweets=results['statuses']
        for tweet in fetched_tweets:
            parsed_tweet={}
            parsed_tweet['text']=tweet['text']
    #print(parsed_tweet['text'])
            parsed_tweet['sentiment']=get_tweet_sentiment(tweet['text'])
    #print(parsed_tweet['sentiment'])
            if tweet['retweet_count'] > 0:
               if parsed_tweet not in tweets:
                    tweets.append(parsed_tweet)
            else:
                tweets.append(parsed_tweet)
    
    try:
        etweets=[tweet for tweet in tweets if tweet['sentiment']== 'excited'] 
        #print("happy tweets percentage: {} %".format(100*len(htweets)/len(tweets)))


    except ZeroDivisionError:
        print("No happy tweets")
    et=int(100*len(etweets)/len(tweets))
    
    try:
        htweets=[tweet for tweet in tweets if tweet['sentiment']== 'happy']
        #print("amused tweets percentage: {} %".format(100*len(atweets)/len(tweets)))



    except ZeroDivisionError:
        print("No amused tweets")
    ht=int(100*len(htweets)/len(tweets))

    try:    
        ntweets=[tweet for tweet in tweets if tweet['sentiment']== 'neutral']
        #print("neutral tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
        ht=int(100*len(htweets)/len(tweets))


    except ZeroDivisionError:
        print("No neutral tweets")
    nt=int(100*len(ntweets)/len(tweets))

    try:    
        stweets=[tweet for tweet in tweets if tweet['sentiment']== 'sad']
        #print("sad tweets percentage: {} %".format(100*len(stweets)/len(tweets)))

    except ZeroDivisionError:
         print("No sad tweets")
    st=int(100*len(stweets)/len(tweets))
    
    try:
        mtweets=[tweet for tweet in tweets if tweet['sentiment']== 'morose']    
        #print("morose tweets percentage: {} %".format(100*len(mtweets)/len(tweets)))

    except ZeroDivisionError:
        print("No morose tweets") 
    mt=int(100*len(mtweets)/len(tweets))
    genre_id=3
    db = pymysql.connect("localhost","root","root","news_articles",charset="utf8")
    cursor = db.cursor()
    sql = "INSERT INTO `articles` (`headline`,`content`,`sentiment_score`,`summary`,`genre_id`,`image_url`,`elated`,`happy`,`neutral`,`sad`,`morose`) VALUES (%s,%s, %s, %s, %s,%s,%s, %s, %s, %s,%s)"
    cursor.execute(sql,(y,content,norm_arti,s_x,genre_id,storepath,et,ht,nt,st,mt))
    print('query commited')
    db.commit()


    db.close()
# disconnect from server

    
#urls_list


0.8500000000000001
query commited
0.25
query commited
0.30000000000000004
query commited
0.7
query commited
0.75
query commited
0.19999999999999996
query commited
-0.25
query commited
0.19999999999999996
query commited
-0.09999999999999998
query commited
1.0499999999999998
query commited
-0.050000000000000044
query commited
0.050000000000000044
query commited
-0.19999999999999996
query commited
0.0
query commited
0.75
query commited
0.3500000000000001
query commited
0.19999999999999996
query commited
0.1499999999999999
query commited
-0.44999999999999996
query commited
0.6499999999999999
query commited
0.050000000000000044
query commited
-0.55


TwythonError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [1]:
#clustering of articles

In [127]:
import nltk, re, pprint
from nltk.tag import pos_tag
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import cluster
from sklearn.cluster import spectral_clustering
import pymysql
tagged_pos_doc_list=[]
concat_tagged_pos_doc_list=[]


In [149]:
db = pymysql.connect("localhost","root","root","news_articles",charset="utf8")
cursor = db.cursor()
sql = "SELECT * from articles where genre_id=2"
cursor.execute(sql)
articles=cursor.fetchall()
news=[]
tagged_pos_doc_list=[]
concat_tagged_pos_doc_list=[]
for article in articles:
    article_dict={'ArticleId': article[0],'Headline':article[1], 'Date':article[3],'Summary':article[7],'Image':article[4] ,'GenreId': article[5],'Content':article[2],'Excited':article[8],'Happy':article[9],'Neutral':article[10],'Sad':article[11],'Morose':article[12]}
    news.append(article_dict)
for i in range(len(news)):
    tagged_sent = pos_tag(news[i]['Content'].split())
    tagged_sent_proper_nouns=[]
    for x in range(len(tagged_sent)):
        if tagged_sent[x][1]=='NNP':
            tagged_sent_proper_nouns.append(tagged_sent[x][0])
        tagged_dict={'tag':news[i]['ArticleId'],'propernoun':tagged_sent_proper_nouns}
    tagged_pos_doc_list.append(tagged_dict)
y=len(tagged_pos_doc_list)
data_set=[]
concat_tagged_pos_doc_list=[]
for items in tagged_pos_doc_list:
    ax=' '.join(items['propernoun'])
    str_list=[]
    str_list.append(ax)
    concat_dict={'tag':items['tag'],'concatstr':str_list}
    concat_tagged_pos_doc_list.append(concat_dict)
concatstr_list=[]
for items in concat_tagged_pos_doc_list:
    concatstr_list.append(items['concatstr'][0])

data_set=''.join(concatstr_list)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(concatstr_list)
X_train_counts.toarray()
similarity_graph = X_train_counts * X_train_counts.T #sentence
similarity_graph.toarray()
z=(int)(y *0.3)
spectral = cluster.SpectralClustering(n_clusters=z,eigen_solver='arpack',affinity="precomputed")
spectral.fit(similarity_graph)
clusters = spectral.fit_predict(similarity_graph)
clusters_list=[]
for i in range(z):
    cluster_no=[]
    for j in range(len(clusters)):
        if clusters[j]==i:
            cluster_no.append(concat_tagged_pos_doc_list[j]['tag'])
    clusters_list.append(cluster_no)
import csv

with open("/home/vaidehi/FlaskApp/cluster/world.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(clusters_list)

/home/vaidehi/anaconda3/lib/python3.5/site-packages/sklearn/manifold/spectral_embedding_.py:217: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


In [129]:
db = pymysql.connect("localhost","root","root","news_articles",charset="utf8")
cursor = db.cursor()
sql = "SELECT * from articles where genre_id=1"
cursor.execute(sql)
articles=cursor.fetchall()
news=[]
tagged_pos_doc_list=[]
concat_tagged_pos_doc_list=[]
for article in articles:
    article_dict={'ArticleId': article[0],'Headline':article[1], 'Date':article[3],'Summary':article[7],'Image':article[4] ,'GenreId': article[5],'Content':article[2],'Excited':article[8],'Happy':article[9],'Neutral':article[10],'Sad':article[11],'Morose':article[12]}
    news.append(article_dict)
for i in range(len(news)):
    tagged_sent = pos_tag(news[i]['Content'].split())
    tagged_sent_proper_nouns=[]
    for x in range(len(tagged_sent)):
        if tagged_sent[x][1]=='NNP':
            tagged_sent_proper_nouns.append(tagged_sent[x][0])
        tagged_dict={'tag':news[i]['ArticleId'],'propernoun':tagged_sent_proper_nouns}
    tagged_pos_doc_list.append(tagged_dict)
y=len(tagged_pos_doc_list)
data_set=[]
concat_tagged_pos_doc_list=[]
for items in tagged_pos_doc_list:
    ax=' '.join(items['propernoun'])
    str_list=[]
    str_list.append(ax)
    concat_dict={'tag':items['tag'],'concatstr':str_list}
    concat_tagged_pos_doc_list.append(concat_dict)
concatstr_list=[]
for items in concat_tagged_pos_doc_list:
    concatstr_list.append(items['concatstr'][0])

data_set=''.join(concatstr_list)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(concatstr_list)
X_train_counts.toarray()
similarity_graph = X_train_counts * X_train_counts.T #sentence
similarity_graph.toarray()
z=(int)(y *0.3)
spectral = cluster.SpectralClustering(n_clusters=z,eigen_solver='arpack',affinity="precomputed")
spectral.fit(similarity_graph)
clusters = spectral.fit_predict(similarity_graph)
clusters_list=[]
for i in range(z):
    cluster_no=[]
    for j in range(len(clusters)):
        if clusters[j]==i:
            cluster_no.append(concat_tagged_pos_doc_list[j]['tag'])
    clusters_list.append(cluster_no)
import csv

with open("/home/vaidehi/FlaskApp/cluster/national.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(clusters_list)


In [101]:
import csv

with open("/home/vaidehi/FlaskApp/cluster/national.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(clusters_list)



In [144]:
db = pymysql.connect("localhost","root","root","news_articles",charset="utf8")
cursor = db.cursor()
sql = "SELECT * from articles where genre_id=3"
cursor.execute(sql)
articles=cursor.fetchall()
news=[]
tagged_pos_doc_list=[]
concat_tagged_pos_doc_list=[]
for article in articles:
    article_dict={'ArticleId': article[0],'Headline':article[1], 'Date':article[3],'Summary':article[7],'Image':article[4] ,'GenreId': article[5],'Content':article[2],'Excited':article[8],'Happy':article[9],'Neutral':article[10],'Sad':article[11],'Morose':article[12]}
    news.append(article_dict)
for i in range(len(news)):
    tagged_sent = pos_tag(news[i]['Content'].split())
    tagged_sent_proper_nouns=[]
    for x in range(len(tagged_sent)):
        if tagged_sent[x][1]=='NNP':
            tagged_sent_proper_nouns.append(tagged_sent[x][0])
        tagged_dict={'tag':news[i]['ArticleId'],'propernoun':tagged_sent_proper_nouns}
    tagged_pos_doc_list.append(tagged_dict)
y=len(tagged_pos_doc_list)
data_set=[]
concat_tagged_pos_doc_list=[]
for items in tagged_pos_doc_list:
    ax=' '.join(items['propernoun'])
    str_list=[]
    str_list.append(ax)
    concat_dict={'tag':items['tag'],'concatstr':str_list}
    concat_tagged_pos_doc_list.append(concat_dict)
concatstr_list=[]
for items in concat_tagged_pos_doc_list:
    concatstr_list.append(items['concatstr'][0])

data_set=''.join(concatstr_list)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(concatstr_list)
X_train_counts.toarray()
similarity_graph = X_train_counts * X_train_counts.T #sentence
similarity_graph.toarray()
z=(int)(y *0.3)
spectral = cluster.SpectralClustering(n_clusters=z,eigen_solver='arpack',affinity="precomputed")
spectral.fit(similarity_graph)
clusters = spectral.fit_predict(similarity_graph)
clusters_list=[]
for i in range(z):
    cluster_no=[]
    for j in range(len(clusters)):
        if clusters[j]==i:
            cluster_no.append(concat_tagged_pos_doc_list[j]['tag'])
    clusters_list.append(cluster_no)
import csv

with open("/home/vaidehi/FlaskApp/cluster/business.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(clusters_list)

In [145]:
db = pymysql.connect("localhost","root","root","news_articles",charset="utf8")
cursor = db.cursor()
sql = "SELECT * from articles where genre_id=4"
cursor.execute(sql)
articles=cursor.fetchall()
news=[]
tagged_pos_doc_list=[]
concat_tagged_pos_doc_list=[]
for article in articles:
    article_dict={'ArticleId': article[0],'Headline':article[1], 'Date':article[3],'Summary':article[7],'Image':article[4] ,'GenreId': article[5],'Content':article[2],'Excited':article[8],'Happy':article[9],'Neutral':article[10],'Sad':article[11],'Morose':article[12]}
    news.append(article_dict)
for i in range(len(news)):
    tagged_sent = pos_tag(news[i]['Content'].split())
    tagged_sent_proper_nouns=[]
    for x in range(len(tagged_sent)):
        if tagged_sent[x][1]=='NNP':
            tagged_sent_proper_nouns.append(tagged_sent[x][0])
        tagged_dict={'tag':news[i]['ArticleId'],'propernoun':tagged_sent_proper_nouns}
    tagged_pos_doc_list.append(tagged_dict)
y=len(tagged_pos_doc_list)
data_set=[]
concat_tagged_pos_doc_list=[]
for items in tagged_pos_doc_list:
    ax=' '.join(items['propernoun'])
    str_list=[]
    str_list.append(ax)
    concat_dict={'tag':items['tag'],'concatstr':str_list}
    concat_tagged_pos_doc_list.append(concat_dict)
concatstr_list=[]
for items in concat_tagged_pos_doc_list:
    concatstr_list.append(items['concatstr'][0])

data_set=''.join(concatstr_list)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(concatstr_list)
X_train_counts.toarray()
similarity_graph = X_train_counts * X_train_counts.T #sentence
similarity_graph.toarray()
z=(int)(y *0.2)
spectral = cluster.SpectralClustering(n_clusters=z,eigen_solver='arpack',affinity="precomputed")
spectral.fit(similarity_graph)
clusters = spectral.fit_predict(similarity_graph)
clusters_list=[]
for i in range(z):
    cluster_no=[]
    for j in range(len(clusters)):
        if clusters[j]==i:
            cluster_no.append(concat_tagged_pos_doc_list[j]['tag'])
    clusters_list.append(cluster_no)
import csv

with open("/home/vaidehi/FlaskApp/cluster/technology.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(clusters_list)

In [138]:
y=len(tagged_pos_doc_list)
tagged_pos_doc_list[0]

{'propernoun': ['Bombay',
  'Stock',
  'Dalal',
  'Street.',
  '(Express',
  'Archive',
  'Photo)',
  'Bombay',
  'Stock',
  'Dalal',
  'Street.',
  '(Express',
  'Archive',
  'Photo)',
  'Sensex',
  'Nifty',
  'BSE',
  'Friday',
  'Nifty',
  'RBI',
  'Vinod',
  'Nair,',
  'Head',
  'Research,',
  'Geojit',
  'Financial',
  'Services.',
  'Wall',
  'Street',
  'US',
  'Emmanuel',
  'Macron’s',
  'Banking',
  'RBI',
  'ICICI',
  'Bank,',
  'Axis',
  'Bank,',
  'SBI,',
  'HDFC',
  'Bank',
  'Shares',
  'Ambuja',
  'Cements',
  'ACC',
  'Lupin',
  'Bharti',
  'Airtel',
  '(1.91',
  'Paints,',
  'ONGC,',
  'Infosys,',
  'Sun',
  'Pharma,',
  'TCS,',
  'Hero',
  'MotoCorp',
  'Sensex',
  'BSE',
  'IT,',
  'Broader',
  'However,',
  'Rs',
  'Globally,',
  'Macron',
  'Key',
  'Japan,',
  'Hong',
  'Kong,',
  'South',
  'Korea',
  'Taiwan',
  'India',
  'News,',
  'Indian',
  'Express',
  'App'],
 'tag': 287}

In [146]:
db = pymysql.connect("localhost","root","root","news_articles",charset="utf8")
cursor = db.cursor()
sql = "SELECT * from articles where genre_id=5"
cursor.execute(sql)
articles=cursor.fetchall()
news=[]
tagged_pos_doc_list=[]
concat_tagged_pos_doc_list=[]
for article in articles:
    article_dict={'ArticleId': article[0],'Headline':article[1], 'Date':article[3],'Summary':article[7],'Image':article[4] ,'GenreId': article[5],'Content':article[2],'Excited':article[8],'Happy':article[9],'Neutral':article[10],'Sad':article[11],'Morose':article[12]}
    news.append(article_dict)
for i in range(len(news)):
    tagged_sent = pos_tag(news[i]['Content'].split())
    tagged_sent_proper_nouns=[]
    for x in range(len(tagged_sent)):
        if tagged_sent[x][1]=='NNP':
            tagged_sent_proper_nouns.append(tagged_sent[x][0])
        tagged_dict={'tag':news[i]['ArticleId'],'propernoun':tagged_sent_proper_nouns}
    tagged_pos_doc_list.append(tagged_dict)
y=len(tagged_pos_doc_list)
data_set=[]
concat_tagged_pos_doc_list=[]
for items in tagged_pos_doc_list:
    ax=' '.join(items['propernoun'])
    str_list=[]
    str_list.append(ax)
    concat_dict={'tag':items['tag'],'concatstr':str_list}
    concat_tagged_pos_doc_list.append(concat_dict)
concatstr_list=[]
for items in concat_tagged_pos_doc_list:
    concatstr_list.append(items['concatstr'][0])

data_set=''.join(concatstr_list)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(concatstr_list)
X_train_counts.toarray()
similarity_graph = X_train_counts * X_train_counts.T #sentence
similarity_graph.toarray()
z=(int)(y *0.3)
spectral = cluster.SpectralClustering(n_clusters=z,eigen_solver='arpack',affinity="precomputed")
spectral.fit(similarity_graph)
clusters = spectral.fit_predict(similarity_graph)
clusters_list=[]
for i in range(z):
    cluster_no=[]
    for j in range(len(clusters)):
        if clusters[j]==i:
            cluster_no.append(concat_tagged_pos_doc_list[j]['tag'])
    clusters_list.append(cluster_no)
import csv

with open("/home/vaidehi/FlaskApp/cluster/lifestyle.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(clusters_list)

In [147]:
db = pymysql.connect("localhost","root","root","news_articles",charset="utf8")
cursor = db.cursor()
sql = "SELECT * from articles where genre_id=6"
cursor.execute(sql)
articles=cursor.fetchall()
news=[]
for article in articles:
    article_dict={'ArticleId': article[0],'Headline':article[1], 'Date':article[3],'Summary':article[7],'Image':article[4] ,'GenreId': article[5],'Content':article[2],'Excited':article[8],'Happy':article[9],'Neutral':article[10],'Sad':article[11],'Morose':article[12]}
    news.append(article_dict)
for i in range(len(news)):
    tagged_sent = pos_tag(news[i]['Content'].split())
    tagged_sent_proper_nouns=[]
    for x in range(len(tagged_sent)):
        if tagged_sent[x][1]=='NNP':
            tagged_sent_proper_nouns.append(tagged_sent[x][0])
        tagged_dict={'tag':news[i]['ArticleId'],'propernoun':tagged_sent_proper_nouns}
    tagged_pos_doc_list.append(tagged_dict)
y=len(tagged_pos_doc_list)
data_set=[]
concat_tagged_pos_doc_list=[]
for items in tagged_pos_doc_list:
    ax=' '.join(items['propernoun'])
    str_list=[]
    str_list.append(ax)
    concat_dict={'tag':items['tag'],'concatstr':str_list}
    concat_tagged_pos_doc_list.append(concat_dict)
concatstr_list=[]
for items in concat_tagged_pos_doc_list:
    concatstr_list.append(items['concatstr'][0])

data_set=''.join(concatstr_list)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(concatstr_list)
X_train_counts.toarray()
similarity_graph = X_train_counts * X_train_counts.T #sentence
similarity_graph.toarray()
z=(int)(y *0.3)
spectral = cluster.SpectralClustering(n_clusters=z,eigen_solver='arpack',affinity="precomputed")
spectral.fit(similarity_graph)
clusters = spectral.fit_predict(similarity_graph)
clusters_list=[]
for i in range(z):
    cluster_no=[]
    for j in range(len(clusters)):
        if clusters[j]==i:
            cluster_no.append(concat_tagged_pos_doc_list[j]['tag'])
    clusters_list.append(cluster_no)
import csv

with open("/home/vaidehi/FlaskApp/cluster/sports.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(clusters_list)

In [148]:
db = pymysql.connect("localhost","root","root","news_articles",charset="utf8")
cursor = db.cursor()
sql = "SELECT * from articles where genre_id=7"
cursor.execute(sql)
articles=cursor.fetchall()
news=[]
tagged_pos_doc_list=[]
concat_tagged_pos_doc_list=[]
for article in articles:
    article_dict={'ArticleId': article[0],'Headline':article[1], 'Date':article[3],'Summary':article[7],'Image':article[4] ,'GenreId': article[5],'Content':article[2],'Excited':article[8],'Happy':article[9],'Neutral':article[10],'Sad':article[11],'Morose':article[12]}
    news.append(article_dict)
for i in range(len(news)):
    tagged_sent = pos_tag(news[i]['Content'].split())
    tagged_sent_proper_nouns=[]
    for x in range(len(tagged_sent)):
        if tagged_sent[x][1]=='NNP':
            tagged_sent_proper_nouns.append(tagged_sent[x][0])
        tagged_dict={'tag':news[i]['ArticleId'],'propernoun':tagged_sent_proper_nouns}
    tagged_pos_doc_list.append(tagged_dict)
y=len(tagged_pos_doc_list)
data_set=[]
concat_tagged_pos_doc_list=[]
for items in tagged_pos_doc_list:
    ax=' '.join(items['propernoun'])
    str_list=[]
    str_list.append(ax)
    concat_dict={'tag':items['tag'],'concatstr':str_list}
    concat_tagged_pos_doc_list.append(concat_dict)
concatstr_list=[]
for items in concat_tagged_pos_doc_list:
    concatstr_list.append(items['concatstr'][0])

data_set=''.join(concatstr_list)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(concatstr_list)
X_train_counts.toarray()
similarity_graph = X_train_counts * X_train_counts.T #sentence
similarity_graph.toarray()
z=(int)(y *0.3)
spectral = cluster.SpectralClustering(n_clusters=z,eigen_solver='arpack',affinity="precomputed")
spectral.fit(similarity_graph)
clusters = spectral.fit_predict(similarity_graph)
clusters_list=[]
for i in range(z):
    cluster_no=[]
    for j in range(len(clusters)):
        if clusters[j]==i:
            cluster_no.append(concat_tagged_pos_doc_list[j]['tag'])
    clusters_list.append(cluster_no)
import csv

with open("/home/vaidehi/FlaskApp/cluster/entertainment.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(clusters_list)